# Grammatical Acceptability Classification by Fine-tuning ConvBERT #

In this exercise, we will fine-tune a ConvBERT model to classify sentences based on their grammatical acceptability using the CoLA (Corpus of Linguistic Acceptability) dataset. The goal is to determine whether a given sentence is grammatically correct (acceptable) or incorrect (unacceptable). The successful implementation of this AI model will greatly enhance the Student Help Centre’s ability to provide valuable, timely assistance to students.



## Section 1: Install and import Required Libraries

In [ ]:
!pip install torch transformers datasets

import torch
from torch.utils.data import DataLoader
from transformers import ConvBertTokenizer, ConvBertForSequenceClassification, AdamW
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score
from torch.nn.utils.rnn import pad_sequence


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


## Section 2: Data Preparation

In [ ]:
# Original code used the CoLA dataset from Hugging Face datasets
# This has been replaced with the SST-2 dataset for reproduction purposes


dataset = load_dataset("glue", "sst2")
train_dataset = dataset['train']
test_dataset = dataset['validation']

# Load ConvBERT Tokenizer
model_name = "YituTech/conv-bert-base"  # Publicly available ConvBERT model from Hugging Face
tokenizer = ConvBertTokenizer.from_pretrained(model_name)

# Tokenize the data
def tokenize(batch):
    return tokenizer(batch['sentence'], padding=True, truncation=True)

train_dataset = train_dataset.map(lambda x: tokenize(x), batched=True)
test_dataset = test_dataset.map(lambda x: tokenize(x), batched=True)

# Convert dataset to PyTorch tensors
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

vocab.txt:   0%|          | 0.00/267k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/872 [00:00<?, ? examples/s]

## Section 3: Model Setup and Fine-tuning

In [ ]:
# Custom Collate Function for Sentence Padding

def collate_fn(batch):
    input_ids = [item['input_ids'] for item in batch]
    attention_mask = [item['attention_mask'] for item in batch]
    labels = [item['label'] for item in batch]

    input_ids_padded = pad_sequence(input_ids, batch_first=True)
    attention_mask_padded = pad_sequence(attention_mask, batch_first=True)
    labels = torch.tensor(labels)

    return {
        'input_ids': input_ids_padded,
        'attention_mask': attention_mask_padded,
        'label': labels
    }

# Load Pre-trained ConvBERT model from Hugging Face in PyTorch
model = ConvBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Setup the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Prepare DataLoader with the custom collate function
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=collate_fn)
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=collate_fn)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Fine-tuning on the CoLA dataset
model.train()
for epoch in range(3):  # train for 3 epochs
    total_loss = 0
    for batch in train_dataloader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")


pytorch_model.bin:   0%|          | 0.00/423M [00:00<?, ?B/s]

Some weights of ConvBertForSequenceClassification were not initialized from the model checkpoint at YituTech/conv-bert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 0.2312
Epoch 2, Loss: 0.1557
Epoch 3, Loss: 0.1279



## Section 4: Evaluate the Model

In [ ]:
# Model Evaluation
model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=-1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(true_labels, predictions)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.8750
